In [1]:

import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()



Saving AWCustomers.csv to AWCustomers.csv


In [3]:
df = pd.read_csv("AWCustomers.csv")
df.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [8]:
df.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'LastUpdated'],
      dtype='object')

In [9]:
# (a) Examine attributes & select relevant ones for predicting bike purchase
# Columns like CustomerID, Name, Email, Phone are identifiers, not useful for prediction.
# After looking at the dataset, these columns are relevant:
selected_features = [
    'BirthDate', 'Gender', 'MaritalStatus', 'Education', 'Occupation',
    'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
    'TotalChildren', 'YearlyIncome'
]
#(b)
df_selected = df[selected_features]
df_selected['BirthDate'] = pd.to_datetime(df_selected['BirthDate'])
current_year = pd.Timestamp.now().year
df_selected['Age'] = current_year - df_selected['BirthDate'].dt.year
df_selected.drop('BirthDate', axis=1, inplace=True)
# (c)
data_types = {
    'Gender': 'Discrete - Nominal',
    'MaritalStatus': 'Discrete - Nominal',
    'Education': 'Discrete - Ordinal',
    'Occupation': 'Discrete - Nominal',
    'HomeOwnerFlag': 'Discrete - Binary Nominal',
    'NumberCarsOwned': 'Discrete - Ratio',
    'NumberChildrenAtHome': 'Discrete - Ratio',
    'TotalChildren': 'Discrete - Ratio',
    'YearlyIncome': 'Continuous - Ratio',
    'Age': 'Continuous - Ratio'
}

print("Selected Data Types:")
for k, v in data_types.items():
    print(f"{k}: {v}")

df_selected.head()


Selected Data Types:
Gender: Discrete - Nominal
MaritalStatus: Discrete - Nominal
Education: Discrete - Ordinal
Occupation: Discrete - Nominal
HomeOwnerFlag: Discrete - Binary Nominal
NumberCarsOwned: Discrete - Ratio
NumberChildrenAtHome: Discrete - Ratio
TotalChildren: Discrete - Ratio
YearlyIncome: Continuous - Ratio
Age: Continuous - Ratio


/tmp/ipython-input-2213324004.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['BirthDate'] = pd.to_datetime(df_selected['BirthDate'])
/tmp/ipython-input-2213324004.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Age'] = current_year - df_selected['BirthDate'].dt.year
/tmp/ipython-input-2213324004.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,Gender,MaritalStatus,Education,Occupation,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,M,M,Bachelors,Clerical,1,3,0,1,81916,38
1,M,M,Partial College,Clerical,1,2,1,2,81076,53
2,F,S,Bachelors,Clerical,0,3,0,0,86387,40
3,M,M,Partial College,Skilled Manual,1,2,1,2,61481,48
4,M,S,Partial College,Skilled Manual,1,1,0,0,51804,50


In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df = pd.read_csv("AWCustomers.csv")
df = df[['BirthDate', 'Gender', 'MaritalStatus', 'Education', 'Occupation',
         'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
         'TotalChildren', 'YearlyIncome']]

#DOb to age conversion
df['BirthDate'] = pd.to_datetime(df['BirthDate'], errors='coerce')
current_year = pd.Timestamp.now().year
df['Age'] = current_year - df['BirthDate'].dt.year
df.drop('BirthDate', axis=1, inplace=True)

#(a)Handle Null values
df = df.dropna()

# (b)Normalization
scaler = MinMaxScaler()
df[['Age', 'YearlyIncome', 'NumberCarsOwned',
    'NumberChildrenAtHome', 'TotalChildren']] = scaler.fit_transform(
    df[['Age', 'YearlyIncome', 'NumberCarsOwned',
        'NumberChildrenAtHome', 'TotalChildren']]
)

#(c)Discretization (Binning)
df['AgeGroup'] = pd.cut(df['Age'], bins=4,
                        labels=['Young', 'Adult', 'Middle Age', 'Senior'])

#(d)Standardization (Z-score)
std_scaler = StandardScaler()
df[['YearlyIncome']] = std_scaler.fit_transform(df[['YearlyIncome']])

#(e)One Hot Encoding
df = pd.get_dummies(df, drop_first=True)

# Final dataset preview
print(df.head())

   HomeOwnerFlag  NumberCarsOwned  NumberChildrenAtHome  TotalChildren  \
0              1              0.6              0.000000       0.333333   
1              1              0.4              0.333333       0.666667   
2              0              0.6              0.000000       0.000000   
3              1              0.4              0.333333       0.666667   
4              1              0.2              0.000000       0.000000   

   YearlyIncome       Age  Gender_M  MaritalStatus_S  \
0      0.298555  0.185714      True            False   
1      0.271180  0.400000      True            False   
2      0.444261  0.214286     False             True   
3     -0.367401  0.328571      True            False   
4     -0.682765  0.357143      True             True   

   Education_Graduate Degree  Education_High School  \
0                      False                  False   
1                      False                  False   
2                      False                  False  

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
#Took first two rows from the processed df for comparison
row1 = df.iloc[0].values.reshape(1, -1)
row2 = df.iloc[1].values.reshape(1, -1)

#(a)Simple Matching Coefficient (SMC)
matches = np.sum(row1 == row2)
smc = matches / len(row1[0])
#Jaccard Similarity (for binary features)
#Convert to boolean for Jaccard
bool1 = row1.astype(bool)
bool2 = row2.astype(bool)
intersection = np.logical_and(bool1, bool2).sum()
union = np.logical_or(bool1, bool2).sum()
jaccard_sim = intersection / union

#Cosine Similarity
cos_sim = cosine_similarity(row1, row2)[0][0]
print("SMC:", round(smc, 4))
print("Jaccard Similarity:", round(jaccard_sim, 4))
print("Cosine Similarity:", round(cos_sim, 4))

#(b)Correlation between NumberCarsOwned and YearlyIncome
df_corr = pd.read_csv("AWCustomers.csv")
df_corr['BirthDate'] = pd.to_datetime(df_corr['BirthDate'], errors='coerce')
df_corr['Age'] = current_year - df_corr['BirthDate'].dt.year
df_corr = df_corr[['NumberCarsOwned', 'YearlyIncome']].dropna()
corr_value = df_corr['NumberCarsOwned'].corr(df_corr['YearlyIncome'])
print("Correlation between NumberCarsOwned and YearlyIncome:", round(corr_value, 4))


SMC: 0.6316
Jaccard Similarity: 0.6667
Cosine Similarity: 0.7304
Correlation between NumberCarsOwned and YearlyIncome: 0.4773
